In [1]:
# Load necessary extensions
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv


# Standard library imports
import sys
import logging
from pathlib import Path

# Third-party library imports
import ipytest
import json

# Local imports
from src.paths import CODE_FOLDER, DATA_DIR, INFERENCE_CODE_FOLDER, PARENT_DIR

sys.path.append(str(PARENT_DIR / CODE_FOLDER))
sys.path.append(str(PARENT_DIR / INFERENCE_CODE_FOLDER))

DATA_FILE_PATH = DATA_DIR / "penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# Prevent SageMaker SDK to log events related to the default
# configuration using the INFO level
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)


In [2]:
LOCAL_MODE = True

In [3]:
import os

bucket = os.getenv("BUCKET")
role = os.getenv("ROLE")

S3_LOCATION = f"s3://{bucket}/penguins"

In [4]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

pipeline_session = PipelineSession(default_bucket=bucket) if not LOCAL_MODE else None

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=bucket),
        "instance_type": "local",
        "image": None
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.m5.xlarge",
        "image": None
    }

In [5]:
import boto3

sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

By default, only the specified columns in transformers are transformed and combined in the output, and the non-specified columns are dropped. (default of 'drop')

## Creating the Preprocessing Script

In [ ]:
%%writefile {CODE_FOLDER}/preprocessor.py
"""
This module preprocesses data for machine learning tasks. It includes functions to read data from CSV files,
split the data into training, validation, and test sets, save baseline data, transform data, and save the
processed data and models.
"""

import os
import tarfile
import tempfile
from pathlib import Path

# Import statements...
from typing import Tuple

import joblib
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


def preprocess(base_directory: str) -> None:
    """
    Preprocess the data by loading, splitting, transforming, saving the splits, and saving the model.

    Args:
        base_directory: The base directory where the input data and outputs will be managed.
    """
    # 1. Load supplied data
    df = _read_data_from_csv_files(base_directory)

    # 2. Split data into train and test sets
    df_train, df_validation, df_test = _split_data(df)

    # 3. Save baseline data
    _save_baselines(base_directory, df_train, df_test)

    # 3. Transform the train and test sets
    target_transformer = ColumnTransformer(
        transformers=[("species", OrdinalEncoder(), [0])]
    )

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="mean"), StandardScaler()
    )

    categorical_transformer = make_pipeline(
        SimpleImputer(strategy="most_frequent"), OneHotEncoder()
    )

    features_transformer = ColumnTransformer(
        transformers=[
            (
                "numeric",
                numeric_transformer,
                make_column_selector(dtype_exclude="object"),
            ),
            ("categorical", categorical_transformer, ["island"]),
        ]
    )

    y_train = target_transformer.fit_transform(
        np.array(df_train.species.values).reshape(-1, 1)
    )
    y_validation = target_transformer.transform(
        np.array(df_validation.species.values).reshape(-1, 1)
    )
    y_test = target_transformer.transform(
        np.array(df_test.species.values).reshape(-1, 1)
    )

    df_train = df_train.drop(columns=["species"], axis=1)
    df_validation = df_validation.drop(columns=["species"], axis=1)
    df_test = df_test.drop(columns=["species"], axis=1)

    X_train = features_transformer.fit_transform(df_train)
    X_validation = features_transformer.transform(df_validation)
    X_test = features_transformer.transform(df_test)

    # 4. Save the train and test splits
    _save_splits(
        base_directory, X_train, y_train, X_validation, y_validation, X_test, y_test
    )

    # 5. Save the model (transformers) in tar.gz format
    _save_model(base_directory, target_transformer, features_transformer)


def _read_data_from_csv_files(base_directory: str) -> pd.DataFrame:
    """
    Read and concatenate data from CSV files located in the input directory.

    Args:
        base_directory: The directory where CSV files are located.

    Returns:
        A DataFrame containing the concatenated data.
    """
    input_directory = Path(base_directory) / "input"
    files = [file for file in input_directory.glob("*.csv")]

    if len(files) == 0:
        raise ValueError(f"No csv files found in {input_directory}")

    raw_data = [pd.read_csv(file) for file in files]
    df = pd.concat(raw_data)

    # Shuffle the data
    return df.sample(frac=1, random_state=42)


def _split_data(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Split the DataFrame into training, validation, and test sets.

    Args:
        df: The DataFrame to be split.

    Returns:
        A tuple containing the training, validation, and test DataFrames.
    """
    df_train, temp = train_test_split(df, test_size=0.3)
    df_validation, df_test = train_test_split(temp, test_size=0.5)

    return df_train, df_validation, df_test


def _save_baselines(
    base_directory: str, df_train: pd.DataFrame = None, df_test: pd.DataFrame = None
) -> None:
    """
    Save baseline versions of the training and test data sets.

    Args:
        base_directory: Directory where the baseline data will be saved.
        df_train: Training data DataFrame.
        df_test: Test data DataFrame.
    """
    for split, data in [("train", df_train), ("test", df_test)]:
        baseline_path = Path(base_directory) / f"{split}-baseline"
        baseline_path.mkdir(parents=True, exist_ok=True)

        df = data.copy().dropna()

        # Save header only for the train baseline
        header = True if split == "train" else False
        df.to_csv(baseline_path / f"{split}-baseline.csv", index=False, header=header)


def _save_splits(
    base_directory: str,
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_validation: np.ndarray,
    y_validation: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> None:
    """
    Save the training, validation, and test sets after concatenating features with their respective targets.

    Args:
        base_directory: Directory where the data splits will be saved.
        X_train: Features of the training set.
        y_train: Target of the training set.
        X_validation: Features of the validation set.
        y_validation: Target of the validation set.
        X_test: Features of the test set.
        y_test: Target of the test set.
    """
    train = np.concatenate((X_train, y_train), axis=1)
    validation = np.concatenate((X_validation, y_validation), axis=1)
    test = np.concatenate((X_test, y_test), axis=1)

    train_path = Path(base_directory) / "train"
    validation_path = Path(base_directory) / "validation"
    test_path = Path(base_directory) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", index=False, header=False)
    pd.DataFrame(validation).to_csv(
        validation_path / "validation.csv", index=False, header=False
    )
    pd.DataFrame(test).to_csv(test_path / "test.csv", index=False, header=False)


def _save_model(base_directory: str, target_transformer, features_transformers) -> None:
    """
    Save the preprocessing model (transformers) in tar.gz format.

    Args:
        base_directory: Directory where the model will be saved.
        target_transformer: The transformer used for the target variable.
        features_transformers: The transformers used for the feature variables.
    """
    with tempfile.TemporaryDirectory() as directory:
        joblib.dump(target_transformer, os.path.join(directory, "target.joblib"))
        joblib.dump(features_transformers, os.path.join(directory, "features.joblib"))

        model_path = Path(base_directory) / "model"
        model_path.mkdir(parents=True, exist_ok=True)

        with tarfile.open(f"{str(model_path / 'model.tar.gz')}", "w:gz") as tar:
            tar.add(os.path.join(directory, "target.joblib"), arcname="target.joblib")
            tar.add(
                os.path.join(directory, "features.joblib"), arcname="features.joblib"
            )


if __name__ == "__main__":
    preprocess(base_directory="/opt/ml/processing")


In [ ]:
%%ipytest -s

import pytest
import tempfile
from pathlib import Path
import shutil
import tarfile
from preprocessor import preprocess


@pytest.fixture(scope="function", autouse=False)
def directory():
    directory = tempfile.mkdtemp()
    
    input_directory = Path(directory) / "input"
    input_directory.mkdir(parents=True, exist_ok=True)

    shutil.copy2(DATA_FILE_PATH, input_directory / "data.csv")

    directory = Path(directory)
    preprocess(directory)

    yield directory
    
    shutil.rmtree(directory)


def test_preprocess_generate_baselines(directory):
    output_directories = os.listdir(directory)

    assert "train-baseline" in output_directories
    assert "test-baseline" in output_directories


def test_preprocess_generate_data_splits(directory):
    output_directories = os.listdir(directory)

    assert "train" in output_directories
    assert "validation" in output_directories
    assert "test" in output_directories


def test_preprocess_creates_two_models(directory):
    model_path = directory / "model"

    tar = tarfile.open(model_path / "model.tar.gz", "r:gz")

    assert "target.joblib" in tar.getnames()
    assert "features.joblib" in tar.getnames()


def tests_splits_are_transformed(directory):
    train = pd.read_csv(directory / "train" / "train.csv", header=None)
    validation = pd.read_csv(directory / "validation" / "validation.csv", header=None)
    test = pd.read_csv(directory / "test" / "test.csv", header=None)

    # The number of features should be 7
    # * 3 - island (one-hot encoded)
    # * 1 - culmen_length_mm
    # * 1 - culmen_depth_mm
    # * 1 - flipper_length_mm
    # * 1 - body_mass_g
    num_features = 7

    # The number of targets should be 1
    assert train.shape[1] == num_features + 1
    assert validation.shape[1] == num_features + 1
    assert test.shape[1] == num_features + 1
    

## Setting up the Processing Step

In [6]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="P15D") # type: ignore

In [7]:
from sagemaker.workflow.parameters import ParameterString

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

In [11]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    framework_version="1.2-1",
    base_job_name="preprocess-data",
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [12]:
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput

preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(
                source=dataset_location,
                destination="/opt/ml/processing/input"
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source="/opt/ml/processing/train",
                destination=f"{S3_LOCATION}/preprocessing/train",
            ),
            ProcessingOutput(
                output_name="validation",
                source="/opt/ml/processing/validation",
                destination=f"{S3_LOCATION}/preprocessing/validation",
            ),
            ProcessingOutput(
                output_name="test",
                source="/opt/ml/processing/test",
                destination=f"{S3_LOCATION}/preprocessing/test",
            ),
            ProcessingOutput(
                output_name="model",
                source="/opt/ml/processing/model",
                destination=f"{S3_LOCATION}/preprocessing/model",
            ),
            ProcessingOutput(
                output_name="train-baseline",
                source="/opt/ml/processing/train-baseline",
                destination=f"{S3_LOCATION}/preprocessing/train-baseline",
            ),
            ProcessingOutput(
                output_name="test-baseline",
                source="/opt/ml/processing/test-baseline",
                destination=f"{S3_LOCATION}/preprocessing/test-baseline",
            ),
        ],
    ),
    cache_config=cache_config,
)

/Users/carlos/Projects/penguin-classifier/.venv/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Creating the Pipeline